In [9]:
using ITensors
using ITensorMPS
using JLD2
using Plots
using LinearAlgebra
using Printf
using Measures 

gr() 

Plots.GRBackend()

In [10]:


function analyse_hpc_results()
    filename = joinpath(@__DIR__, "all_MPS_data_compressed.jld2")
    if !isfile(filename)
        println("Error: $filename not found.")
        return
    end
    
    results_db = load(filename, "results_db")
    
    # Collect available N values
    Ns = sort(unique([k[1] for k in keys(results_db)]))
    
    println("Hypothesis: The high bond dimension in noisy systems is an artifact.")
    
    for N in Ns
        if !haskey(results_db, (N, 0.0))
            println("Skipping N=$N (Missing baseline data)")
            continue
        end
        
        clean_data = results_db[(N, 0.0)]
        
        chi_clean = clean_data["maxlinkdim"]
        
        # println("\nSystem Size N = $N (Clean BD = $chi_clean)")
        # println("-"^60)
        # @printf("%-10s | %-12s | %-12s | %-10s\n", "Sigma", "Noisy BD", "Truncated F", "Result")
        # println("-"^60)
        
        noisy_sigmas = sort([k[2] for k in keys(results_db) if k[1] == N && k[2] > 0.0])
        
        for σ in noisy_sigmas
            noisy_data = results_db[(N, σ)]
            
            
            chi_noisy = noisy_data["maxlinkdim"]
            fidelity = noisy_data["fidelity"]
            
            status = fidelity > 0.99 ? "PASS" : "FAIL"
            
            # @printf("%-10.3f | %-12d | %-12.6f | %-10s\n", σ, chi_noisy, fidelity, status)
        end
        println("-"^60)
    end
    

    
    plot_list = []
    
    for N in Ns
        # Get sigmas specifically for this N
        sigmas = sort([k[2] for k in keys(results_db) if k[1] == N])
        
        # Create individual plot for this N
        p = plot(title="N=$N", 
                 xlabel="Index", ylabel="Prob (λ²)",
                 yaxis=:log10, 
                 legend=:topright, 
                 legendfontsize=5,
                 titlefontsize=10,
                 guidefontsize=7,
                 tickfontsize=6)
                 
        for σ in sigmas
            spec = results_db[(N, σ)]["spectrum"]
            # Sort descending
            spec = sort(spec, rev=true)
            
            # Shorten labels to save space
            lbl = σ == 0.0 ? "Clean" : "σ=$σ"
            ls = σ == 0.0 ? :solid : :dash
            
            plot!(p, spec, label=lbl, linestyle=ls, lw=1.5)
        end
        
        push!(plot_list, p)
    end
    
    # Calculate grid dimensions
    n_plots = length(plot_list)
    n_cols = 3  
    n_rows = ceil(Int, n_plots / n_cols)
    
    img_height = n_rows * 350
    img_width = 1200
    
    # Combine all plots into one layout
    final_plot = plot(plot_list..., 
                      layout=(n_rows, n_cols), 
                      size=(img_width, img_height),
                      dpi=300,
                      margin=5mm) 
    
    output_filename = joinpath(@__DIR__, "all_spectra_combined.png")
    savefig(final_plot, output_filename)

    return results_db
end

results_db = analyse_hpc_results()

Hypothesis: The high bond dimension in noisy systems is an artifact.
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
----------------

Dict{Any, Any} with 140 entries:
  (28, 0.05)  => Dict{String, Any}("sigma"=>0.05, "N"=>28, "maxlinkdim"=>2243, …
  (34, 0.01)  => Dict{String, Any}("sigma"=>0.01, "N"=>34, "maxlinkdim"=>1324, …
  (40, 0.02)  => Dict{String, Any}("sigma"=>0.02, "N"=>40, "maxlinkdim"=>2436, …
  (2, 0.01)   => Dict{String, Any}("sigma"=>0.01, "N"=>2, "maxlinkdim"=>2, "fid…
  (10, 0.01)  => Dict{String, Any}("sigma"=>0.01, "N"=>10, "maxlinkdim"=>32, "f…
  (12, 0.001) => Dict{String, Any}("sigma"=>0.001, "N"=>12, "maxlinkdim"=>36, "…
  (14, 0.005) => Dict{String, Any}("sigma"=>0.005, "N"=>14, "maxlinkdim"=>92, "…
  (18, 0.01)  => Dict{String, Any}("sigma"=>0.01, "N"=>18, "maxlinkdim"=>212, "…
  (26, 0.01)  => Dict{String, Any}("sigma"=>0.01, "N"=>26, "maxlinkdim"=>658, "…
  (28, 0.001) => Dict{String, Any}("sigma"=>0.001, "N"=>28, "maxlinkdim"=>177, …
  (36, 0.05)  => Dict{String, Any}("sigma"=>0.05, "N"=>36, "maxlinkdim"=>5686, …
  (34, 0.002) => Dict{String, Any}("sigma"=>0.002, "N"=>34, "maxlinkdim"=>26

In [11]:
function visualise_verification_metrics(results_db)
    
    Ns = sort(unique([k[1] for k in keys(results_db)]))
    
    # Bond Dimension Growth (Log-Log to capture scaling)
    p_bd = plot(title="Bond Dimension Growth with Noise", 
                xlabel="Noise Strength (σ)", 
                ylabel="Max Bond Dimension (χ)",
                xscale=:log10, yscale=:log10,
                legend=:outertopright,
                legendfontsize=6,
                size=(800, 500), dpi=300, margin=5mm)

    # Infidelity (1 - F)
    # We plot (1 - F) on a log scale because F is very close to 1.0.
    # This makes the tiny deviations (0.9999...) visible.
    p_fid = plot(title="Truncation Error (Artifact Check)", 
                 xlabel="Noise Strength (σ)", 
                 ylabel="Infidelity (1 - Fidelity)",
                 xscale=:log10, yscale=:log10,
                 legend=:outertopright,
                 legendfontsize=6,
                 size=(800, 500), dpi=300, margin=5mm)

    for N in Ns
        # Filter and sort keys for this N
        n_keys = [k for k in keys(results_db) if k[1] == N && k[2] > 0.0]
        sort!(n_keys, by = k -> k[2]) # Sort by sigma
        
        if isempty(n_keys) continue end
        
        # Extract vectors
        sigmas = [k[2] for k in n_keys]
        bds = [results_db[k]["maxlinkdim"] for k in n_keys]
        fids = [results_db[k]["fidelity"] for k in n_keys]
        
        # Convert Fidelity to Infidelity for Log Plot (ensure no log(0))
        infidelities = [max(1.0 - f, 1e-16) for f in fids]

        # Add lines to plots
        plot!(p_bd, sigmas, bds, label="N=$N", lw=2, marker=:circle, markersize=3)
        plot!(p_fid, sigmas, infidelities, label="N=$N", lw=2, marker=:square, markersize=3)
    end
    
    # Save the files
    savefig(p_bd, "verification_bond_dimension.png")
    savefig(p_fid, "verification_infidelity.png")
    

end


visualise_verification_metrics(results_db)

"c:\\Users\\Ethan\\OneDrive - University of Bristol\\Labs\\Y4 Labs\\Code_for_git\\Ethan_code\\Machine_prec\\All_MPS_data_analysis\\verification_infidelity.png"

In [25]:
# Detect Divergence & Plot Individual Spectra with Cutoffs
function analyse_divergence_points(results_db, N_target; threshold_ratio=10.0)
    
    # 1. Get Clean Baseline
    if !haskey(results_db, (N_target, 0.0))
        println("Error: No clean baseline for N=$N_target")
        return Dict()
    end
    
    clean_spec = results_db[(N_target, 0.0)]["spectrum"]
    # Ensure sorted descending
    sort!(clean_spec, rev=true)
    
    # 2. Get Sigmas
    sigmas = sort([k[2] for k in keys(results_db) if k[1] == N_target && k[2] > 0.0])
    
    cutoffs = Dict()
    plot_list = []
    
    for σ in sigmas
        noisy_spec = results_db[(N_target, σ)]["spectrum"]
        sort!(noisy_spec, rev=true)
        
        # --- FIND CUTOFF INDEX ---
        # We compare noisy vs clean. 
        # Cutoff is where Noisy > Threshold * Clean 
        # OR where Clean runs out of indices.
        cutoff_idx = length(clean_spec) # Default to end of clean
        
        min_len = min(length(clean_spec), length(noisy_spec))
        
        for i in 1:min_len
            val_clean = clean_spec[i]
            val_noisy = noisy_spec[i]
            
            # Avoid division by zero
            if val_clean < 1E-20 
                cutoff_idx = i
                break
            end
            
            # Check for divergence (Ratio check)
            if val_noisy > (val_clean * threshold_ratio)
                cutoff_idx = i
                break
            end
        end
        
        cutoffs[σ] = cutoff_idx
        
        # --- INDIVIDUAL PLOT FOR THIS SIGMA ---
        p = plot(title="σ=$σ (Cutoff: $cutoff_idx)", 
                 xlabel="Index", ylabel="Prob (λ²)",
                 yaxis=:log10, legend=:topright,
                 titlefontsize=10, guidefontsize=8, tickfontsize=7)
        
        # Plot Clean (Reference)
        plot!(p, clean_spec, label="Clean", lc=:black, lw=1, alpha=0.5)
        # Plot Noisy
        plot!(p, noisy_spec, label="Noisy", lc=:blue, lw=2)
        # Add Vertical Line at Cutoff
        vline!(p, [cutoff_idx], label="Div.", lc=:red, ls=:dash)
        
        push!(plot_list, p)
    end
    
    n_plots = length(plot_list)
    n_cols = 3
    n_rows = ceil(Int, n_plots / n_cols)
    
    final_plot = plot(plot_list..., layout=(n_rows, n_cols), 
                      size=(1000, n_rows * 300), dpi=300, margin=5mm)
    
    filename = "divergence_for_N$(N_target).png"
    savefig(final_plot, filename)
    
    # return cutoffs
end

analyse_divergence_points(results_db, 30; threshold_ratio=10.0)

    

"c:\\Users\\Ethan\\OneDrive - University of Bristol\\Labs\\Y4 Labs\\Code_for_git\\Ethan_code\\Machine_prec\\All_MPS_data_analysis\\divergence_for_N30.png"

In [5]:

# Calculate & Plot Tail Area vs N

function analyse_tail_areas(results_db)
    
    Ns = sort(unique([k[1] for k in keys(results_db)]))
    Ns = (20:2:36)
    
    # Structure: tails[sigma] = [ (N, Area), (N, Area), ... ]
    tails_by_sigma = Dict()
    
    # Pre-load all clean spectra to avoid repeated lookups
    clean_specs = Dict()
    for N in Ns
        if haskey(results_db, (N, 0.0))
            clean_specs[N] = sort(results_db[(N, 0.0)]["spectrum"], rev=true)
        end
    end
    
    for (key, data) in results_db
        N, σ = key
        if σ == 0.0 continue end # Skip clean itself
        if !haskey(clean_specs, N) continue end
        
        noisy_spec = sort(data["spectrum"], rev=true)
        clean_spec = clean_specs[N]
        
        cutoff_idx = length(clean_spec)
        min_len = min(length(clean_spec), length(noisy_spec))
        
        for i in 1:min_len
            if clean_spec[i] < 1E-20 || noisy_spec[i] > (clean_spec[i] * 10.0)
                cutoff_idx = i
                break
            end
        end
        
        # Sum of probabilities from cutoff to end
        if cutoff_idx < length(noisy_spec)
            tail_area = sum(noisy_spec[cutoff_idx:end])
        else
            tail_area = 0.0
        end
        
        # Store
        if !haskey(tails_by_sigma, σ)
            tails_by_sigma[σ] = []
        end
        push!(tails_by_sigma[σ], (N, tail_area))
    end
    
    p = plot(title="Tail Weight vs System Size", 
             xlabel="System Size (N)", 
             ylabel="Total Tail Probability",
             legend=:outertopright, marker=:circle,
             size=(800, 500), dpi=300)
             
    sigmas = sort(collect(keys(tails_by_sigma)))
    
    for σ in sigmas
        data_points = tails_by_sigma[σ]
        sort!(data_points, by = x -> x[1]) # Sort by N
        
        X = [x[1] for x in data_points]
        Y = [x[2] for x in data_points]
        
        plot!(p, X, Y, label="σ=$σ", lw=2)
    end
    
    savefig(p, "tail_area_vs_N_20.png")
end

analyse_tail_areas(results_db)

"c:\\Users\\Ethan\\OneDrive - University of Bristol\\Labs\\Y4 Labs\\Code_for_git\\Ethan_code\\Machine_prec\\All_MPS_data_analysis\\tail_area_vs_N_20.png"

In [6]:


# Noise Scaling Verification

function analyse_noise_scaling(results_db)
    println("Reason: To determine if the 'Tail' is simple perturbative noise (slope ~2) or something more complex.")
    
    Ns = sort(unique([k[1] for k in keys(results_db)]))
    Ns = [4,10,16,24,30,32,34,36]
    
    p = plot(title="Tail Area Scaling with Noise Strength", 
             xlabel="Noise Strength (σ)", 
             ylabel="Tail Area (Weight)",
             xscale=:log10, yscale=:log10,
             legend=:bottomright,
             size=(800, 500), dpi=300)
    
    # Reference Line for Slope 2 (Perturbation Theory Prediction)
    # y = c * x^2  => log(y) = 2*log(x) + c
    ref_x = [1e-3, 5e-2]
    ref_y = [1e-7, 1e-7 * (5e-2/1e-3)^2] # Scale by square
    plot!(p, ref_x, ref_y, label="Slope 2 Ref (Perturbative)", color=:gray, ls=:dash)
    
    for N in Ns
        # Get data for this N
        n_keys = [k for k in keys(results_db) if k[1] == N && k[2] > 0.0]
        sort!(n_keys, by = k -> k[2])
        
        if isempty(n_keys) continue end
        
        # Calculate Tail Areas again (or pass from prev function)
        sigmas = []
        areas = []
        
        clean_spec = results_db[(N, 0.0)]["spectrum"]
        sort!(clean_spec, rev=true)
        
        for k in n_keys
            σ = k[2]
            noisy_spec = sort(results_db[k]["spectrum"], rev=true)
            
            # Recalculate Cutoff
            cutoff_idx = length(clean_spec)
            min_len = min(length(clean_spec), length(noisy_spec))
            for i in 1:min_len
                if clean_spec[i] < 1E-20 || noisy_spec[i] > (clean_spec[i] * 10.0)
                    cutoff_idx = i
                    break
                end
            end
            
            val = (cutoff_idx < length(noisy_spec)) ? sum(noisy_spec[cutoff_idx:end]) : 1e-16
            
            push!(sigmas, σ)
            push!(areas, val)
        end
        
        plot!(p, sigmas, areas, label="N=$N", marker=:circle, markersize=3)
    end
    
    savefig(p, "tail_scaling_analysis.png")
end

analyse_noise_scaling(results_db)

Reason: To determine if the 'Tail' is simple perturbative noise (slope ~2) or something more complex.


"c:\\Users\\Ethan\\OneDrive - University of Bristol\\Labs\\Y4 Labs\\Code_for_git\\Ethan_code\\Machine_prec\\All_MPS_data_analysis\\tail_scaling_analysis.png"